In [1]:
# ⬇️ Clean install of Chromium, Chromedriver and Selenium
!apt-get update
!apt install -y chromium-browser chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/chromedriver
!pip install --upgrade selenium

# ✅ Set PATH for Colab to find Chrome
import os
os.environ['PATH'] += ":/usr/lib/chromium-browser/:/usr/bin/"


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,161 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,066 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64

In [18]:
!pip install webdriver-manager

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time


options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)


# Target URL
url = "https://laws.moj.gov.sa/ar/JudicialDecisionsList/3"
driver.get(url)
time.sleep(5)

data = []

while True:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    cards = soup.find_all('div', class_='legislation-card')

    for card in cards:
        try:
            # Judgment number
            judgment_div = card.find('div', class_='h4 pointer ellipsis')
            judgment_number = judgment_div.get_text(strip=True).split('/')[-1]


            # Details link
            link_tag = card.find('a', class_='details-link')
            detail_link = "https://laws.moj.gov.sa" + link_tag['href'] if link_tag else "N/A"

            # Append record
            data.append({
                'Judgment Number': judgment_number,
                'Details Link': detail_link
            })

        except Exception as e:
            print("❌ Error parsing a card:", e)

    # Pagination: Try clicking the "Next" button
    try:
        next_btn = driver.find_element("xpath", "//a[contains(@aria-label, 'Next')]")
        if "disabled" in next_btn.get_attribute("class"):
            break  # Reached last page
        next_btn.click()
        time.sleep(5)
    except Exception:
        break  # No more pages or button not found

driver.quit()

# Save results to CSV
df = pd.DataFrame(data)
df.to_csv("ksa_judgments.csv", index=False, encoding="utf-8-sig")

print(f"✅ Scraping complete. Total judgments extracted: {len(df)}")

✅ Scraping complete. Total judgments extracted: 12


In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup headless Chrome browser
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

# Open the starting page
base_url = "https://laws.moj.gov.sa/ar/JudicialDecisionsList/3"
driver.get(base_url)

data = []
page_number = 1

while True:
    print(f"🔄 Scraping page {page_number}...")

    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "legislation-card")))
    except Exception as e:
        print(f"❌ Timeout waiting for cards on page {page_number}: {e}")
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    cards = soup.find_all('div', class_='legislation-card')

    if not cards:
        print(f"⚠️ No cards found on page {page_number}.")
        break

    for card in cards:
        try:
            judgment_div = card.find('div', class_='h4 pointer ellipsis')
            judgment_number = judgment_div.get_text(strip=True).split('/')[-1]

            link_tag = card.find('a', class_='details-link')
            detail_link = "https://laws.moj.gov.sa" + link_tag['href'] if link_tag else "N/A"

            data.append({
                'Judgment Number': judgment_number,
                'Details Link': detail_link
            })
        except Exception as e:
            print("❌ Error parsing a card:", e)

    # Debug: list all pagination buttons on the current page
    buttons = driver.find_elements(By.XPATH, "//button[contains(@class, 'page-link')]")
    print(f"🧪 Found {len(buttons)} pagination buttons on page {page_number}:")
    for b in buttons:
        aria = b.get_attribute("aria-label")
        text = b.text
        disabled = b.get_attribute("disabled")
        classes = b.get_attribute("class")
        print(f" - Text: '{text}', aria-label: '{aria}', disabled: {disabled}, class: '{classes}'")

    # Try clicking the "Next" (التالي) button using aria-label
    try:
        next_button = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//button[@aria-label='Go to next page' and contains(@class, 'page-link')]"
        )))
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        # Try JS click as fallback for better reliability
        driver.execute_script("arguments[0].click();", next_button)
        page_number += 1
        time.sleep(2)
    except Exception as e:
        print("✅ No more pages or 'التالي' button not found or click failed.")
        break

driver.quit()

df = pd.DataFrame(data)
df.to_csv("ksa_judgments.csv", index=False, encoding="utf-8-sig")

print(f"✅ Done! Total judgments extracted: {len(df)}")


🔄 Scraping page 1...
🧪 Found 6 pagination buttons on page 1:
 - Text: '1', aria-label: 'Go to page 1', disabled: None, class: 'page-link'
 - Text: '2', aria-label: 'Go to page 2', disabled: None, class: 'page-link'
 - Text: '3', aria-label: 'Go to page 3', disabled: None, class: 'page-link'
 - Text: '4', aria-label: 'Go to page 4', disabled: None, class: 'page-link'
 - Text: 'التالي', aria-label: 'Go to next page', disabled: None, class: 'page-link'
 - Text: 'الأخيرة', aria-label: 'Go to last page', disabled: None, class: 'page-link'
🔄 Scraping page 2...
🧪 Found 8 pagination buttons on page 2:
 - Text: 'الأولى', aria-label: 'Go to first page', disabled: None, class: 'page-link'
 - Text: 'السابق', aria-label: 'Go to previous page', disabled: None, class: 'page-link'
 - Text: '1', aria-label: 'Go to page 1', disabled: None, class: 'page-link'
 - Text: '2', aria-label: 'Go to page 2', disabled: None, class: 'page-link'
 - Text: '3', aria-label: 'Go to page 3', disabled: None, class: 'page